In [1]:
pip install pytelegrambotapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.3/263.3 kB 5.7 MB/s eta 0:00:00


In [10]:
import telebot
import math
token = ''

bot = telebot.TeleBot(token)

@bot.message_handler(commands=["start", "help"])
def begin(message):
  bot.send_message(message.chat.id,
                   """/add - добавить дело\n/check - проверить список дел\n/remove - удалить выполненное дело\n/help - вывести список команд""")

@bot.message_handler(commands=["add"])
def addition(message):
  sent1 = bot.send_message(message.chat.id, "Что вы хотите добавить?")
  bot.register_next_step_handler(sent1, add_to_file)

def add_to_file(message):
  file = str(message.chat.id)+".txt"
  with open(file, 'a', encoding='utf-8') as f:
    f.write(str(message.text)+'\n')
    f.close()
  bot.send_message(message.chat.id, str(message.text)+" добавлено")


@bot.message_handler(commands=["check"])
def check_text(message):
  sent2 = bot.send_message(message.chat.id, "Список дел:")
  file = str(message.from_user.id) + '.txt'

  try:
    f = open(file, 'r', encoding='utf-8')
  except Exception:
    f = open(file, 'w', encoding='utf-8')
  finally:
    lines = open(file, 'r', encoding='utf-8').readlines()
    b = ""
    for i in lines:
      b = b + str(lines.index(i))+ ' ' + i + '\n'
    bot.send_message(message.chat.id, str(b))


@bot.message_handler(commands=["remove"])
def remove_text(message):
  sent3 = bot.send_message(message.chat.id, 'Введите номер строки для удаления')
  bot.register_next_step_handler(sent3, remove_rep)

def remove_rep(message):
  file = str(message.from_user.id) + '.txt'

  try:
    number = math.fabs(int(message.text))
  except Exception:
    bot.send_message(message.chat.id, 'Вы ввели не положительное число')
  else:
    replace_line(file, number-1, "")
    bot.send_message(message.chat.id, str(message.text) + ' удалено')

def replace_line(file_name, line_num, text):
  lines = open(file_name, 'r').readlines()
  if line_num in list(range(len(lines))):
    lines[line_num] = text
    out = open(file_name, 'w')
    out.writelines(lines)
    out.close()


bot.polling(non_stop=True, interval=0)